## 1) Create a new notebook, import the relevant libraries & ords_prods_merge dataframe

## Script Contents
### Grouping
### Aggregating
### Creating flags

In [1]:
# Importing libraries and data

import pandas as pd
import numpy as np
import os

In [2]:
path = r'C:\Users\anon\Documents\CareerFoundry\Data Analytics Immersion\Instacart Basket Analysis'

In [3]:
ords_prods_merge = pd.read_pickle(os.path.join(path, 'Data', 'Prepared Data', 'orders_products_merged.pkl'))

In [4]:
depts = pd.read_csv(os.path.join(path, 'Data', 'Prepared Data', 'departments_wrangled.csv'))

In [5]:
depts.shape

(21, 2)

In [6]:
depts

,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol
5,6,international
6,7,beverages
7,8,pets
8,9,dry goods pasta
9,10,bulk


In [7]:
ords_prods_merge.shape

(32404859, 14)

In [8]:
ords_prods_merge.head(10)

,order_id,user_id,order_number,orders_day_of_the_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,_merge
0,2539329,1,1,2,8,NaN,196,1,0,Soda,77,7,9.0,both
1,2398795,1,2,3,7,15.0,196,1,1,Soda,77,7,9.0,both
2,473747,1,3,3,12,21.0,196,1,1,Soda,77,7,9.0,both
3,2254736,1,4,4,7,29.0,196,1,1,Soda,77,7,9.0,both
4,431534,1,5,4,15,28.0,196,1,1,Soda,77,7,9.0,both
5,3367565,1,6,2,7,19.0,196,1,1,Soda,77,7,9.0,both
6,550135,1,7,1,9,20.0,196,1,1,Soda,77,7,9.0,both
7,3108588,1,8,1,14,14.0,196,2,1,Soda,77,7,9.0,both
8,2295261,1,9,1,16,0.0,196,4,1,Soda,77,7,9.0,both
9,2550362,1,10,4,8,30.0,196,1,1,Soda,77,7,9.0,both


## 2) Find the aggregated mean of the entire dataframe

In [9]:
# Group by 'product_name'

ords_prods_merge.groupby('product_name')

In [10]:
# Apply agg function to each group to get mean values for selected column

ords_prods_merge.groupby('department_id').agg({'order_number': ['mean']})

,order_number
,mean
department_id,
1,15.457838
2,17.277920
3,17.170395
4,17.811403
5,15.215751
6,16.439806
7,17.225802
8,15.340650


## 3) Analyze the result. How do the results for the entire dataframe differ from those of the subset?

#### The results of aggregating the mean by department for the entire dataframe show results that have less variability than when looking at a subset. It is interesting to note that entire departments were excluded from the subset, which demonstrates that subsets are not necessarily representative population samples of a dataframe.

## 4) Create a loyalty flag for existing customers using the transform() and loc() functions.

In [11]:
# Create a loyalty tag

ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)

In [12]:
#Setting a loyalty flag using 'loc()'

ords_prods_merge.loc[ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'

In [13]:
ords_prods_merge.loc[(ords_prods_merge['max_order'] <= 40) & (ords_prods_merge['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'

In [14]:
ords_prods_merge.loc[ords_prods_merge['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [15]:
ords_prods_merge['loyalty_flag'].value_counts(dropna = False)

Regular customer    15876776
Loyal customer      10284093
New customer         6243990
Name: loyalty_flag, dtype: int64

## 5) Use the loyalty flag and check the basic statistics of the product prices for each loyalty category to determine whether the prices of products purchased by loyal customers differ from those purchased by regular or new customers

In [16]:
ords_prods_merge.groupby('loyalty_flag').agg({'prices': ['mean', 'min', 'max']})

prices              
                       mean  min      max
loyalty_flag                             
Loyal customer    10.386336  1.0  99999.0
New customer      13.294670  1.0  99999.0
Regular customer  12.495717  1.0  99999.0

#### Need to investigate if the max price is valid - 99999 is unreasonably high for a grocery item

In [17]:
# Check price list

ords_prods_merge['prices'].drop_duplicates().sort_values(ascending = False)

29165516    99999.0
10030345    14900.0
18504754       25.0
25758883       24.9
28425432       24.8
             ...   
3384024         1.4
632766          1.3
5488887         1.2
5147325         1.1
60725           1.0
Name: prices, Length: 242, dtype: float64

In [18]:
# look for items prices at 99999 or 14900

high_price_items = ords_prods_merge[ords_prods_merge['prices'].isin([99999, 14900])]

In [19]:
# Find the names of the strangely priced products

high_price_items['product_name'].drop_duplicates().sort_values(ascending = False)

10030345    Lowfat 2% Milkfat Cottage Cheese
29165516               2 % Reduced Fat  Milk
Name: product_name, dtype: object

In [20]:
# Check the average price of cottage cheese when not including 'Lowfat 2% Milkfat Cottage Cheese'

c_cheese = ords_prods_merge[
    (ords_prods_merge['product_name'].str.contains('Cottage Cheese', case=False)) &
    ~(ords_prods_merge['product_name'] == 'Lowfat 2% Milkfat Cottage Cheese')
]

In [21]:
c_cheese['product_name'].drop_duplicates().sort_values(ascending = False)

31328377                 Strawberry 2% Milkfat Cottage Cheese
16190405          Small Curd Lowfat 2% Milkfat Cottage Cheese
28230189                    Small Curd Low Fat Cottage Cheese
31938390                         Small Curd Cottage Cheese 4%
24919750                            Small Curd Cottage Cheese
                                  ...                        
29604718                        2% Cottage Cheese, Small Curd
28984734    1.5% Milkfat Grade A Pasteurized Lowfat Cottag...
25426732                     1% Milkfat Lowfat Cottage Cheese
30299206                             1% Lowfat Cottage Cheese
32159201                            1% Low Fat Cottage Cheese
Name: product_name, Length: 70, dtype: object

In [22]:
c_cheese['prices'].mean()

7.542921028559814

In [23]:
# Trying to filter for products similar to '2 % Reduced Fat  Milk' in order to find average price and impute price of '2 % Reduced Fat  Milk'

filtered_df = ords_prods_merge[(ords_prods_merge['product_name'].str.contains(' Milk', case=False)) &
                              ((ords_prods_merge['product_name'].str.contains('Whole', case=False)) |
                               (ords_prods_merge['product_name'].str.contains('1%', case=False))) &
                              (~ords_prods_merge['product_name'].str.contains('Milkfat', case=False))]

In [24]:
filtered_df['product_name'].drop_duplicates().sort_values(ascending = False)

27365618    YoTot Apple Sweet Potato with Fruit & Veggies ...
27565037    Yo Toddler Organic Strawberry Banana Whole Mil...
20280949       Yo Baby Organic Whole Milk Banana Mango Yogurt
32352898                    With Whole Almonds Milk Chocolate
26245822       Wildberries Organic Probiotic Whole Milk Kefir
                                  ...                        
15207193                                       1% Lowfat Milk
30396198                             1% Lowfat Chocolate Milk
24035957                                      1% Low Fat Milk
29057281                            1% Low Fat Chocolate Milk
20244349                                    1% Chocolate Milk
Name: product_name, Length: 167, dtype: object

#### It appears the prices of 'Lowfat 2% Milkfat Cottage Cheese' and '2 % Reduced Fat  Milk' are likely typos. The options in this case are to impute or exclude the values of these products. 
#### In this case, I will be using the average price of Cottage Cheese products to replace the price of 'Lowfat 2% Milkfat Cottage Cheese'.
#### It is difficult to filter and find the average prices of milk products similar to '2 % Reduced Fat  Milk' to impute the price (see above), but it seems likely that the decimal in this pricing was misplaced, and therefore I will re-price this item to 9.9

In [25]:
# Updating the price of 'Lowfat 2% Milkfat Cottage Cheese'

c_cheese = ords_prods_merge['product_name'] == 'Lowfat 2% Milkfat Cottage Cheese'

In [26]:
ords_prods_merge.loc[c_cheese, 'prices'] = 7.5

In [27]:
ords_prods_merge.loc[ords_prods_merge['product_name'] == 'Lowfat 2% Milkfat Cottage Cheese']

,order_id,user_id,order_number,orders_day_of_the_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,_merge,max_order,loyalty_flag
10030345,912404,17,12,2,14,5.0,21553,5,0,Lowfat 2% Milkfat Cottage Cheese,108,16,7.5,both,40,Regular customer
10030346,603376,17,22,6,16,4.0,21553,3,1,Lowfat 2% Milkfat Cottage Cheese,108,16,7.5,both,40,Regular customer
10030347,3264360,135,2,2,21,13.0,21553,6,0,Lowfat 2% Milkfat Cottage Cheese,108,16,7.5,both,4,New customer
10030348,892534,135,3,0,8,12.0,21553,3,1,Lowfat 2% Milkfat Cottage Cheese,108,16,7.5,both,4,New customer
10030349,229704,342,8,1,19,30.0,21553,9,0,Lowfat 2% Milkfat Cottage Cheese,108,16,7.5,both,16,Regular customer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10034769,3172853,205650,18,1,9,7.0,21553,17,1,Lowfat 2% Milkfat Cottage Cheese,108,16,7.5,both,25,Regular customer
10034770,2504315,205818,3,5,15,3.0,21553,13,0,Lowfat 2% Milkfat Cottage Cheese,108,16,7.5,both,25,Regular customer
10034771,1108388,205818,5,4,5,1.0,21553,5,1,Lowfat 2% Milkfat Cottage Cheese,108,16,7.5,both,25,Regular customer
10034772,1916142,206049,1,2,17,NaN,21553,2,0,Lowfat 2% Milkfat Cottage Cheese,108,16,7.5,both,5,New customer


In [28]:
milk = ords_prods_merge['product_name'] == '2 % Reduced Fat  Milk'

In [29]:
ords_prods_merge.loc[milk, 'prices'] = 9.9

In [30]:
# Re-xamining data

ords_prods_merge.groupby('department_id').agg({'prices': ['mean', 'min', 'max']})

prices           
                    mean  min   max
department_id                      
1               7.736553  1.0  15.0
2               6.990934  1.1  15.0
3               7.853809  1.0  15.0
4               7.981708  1.0  15.0
5               8.143701  1.0  15.0
6               7.682264  1.0  15.0
7               7.680502  1.0  15.0
8               7.889070  1.0  15.0
9               7.350283  1.0  15.0
10              8.348960  1.4  14.1
11              7.996769  1.0  15.0
12             16.294443  8.0  25.0
13              8.009443  1.0  20.0
14              8.028963  1.0  14.9
15              7.539976  1.0  15.0
16              8.334573  1.0  15.0
17              7.384058  1.0  15.0
18              7.638380  1.0  15.0
19              4.275437  1.6   7.0
20              7.778701  1.0  15.0
21              8.660716  1.0  15.0

In [31]:
ords_prods_merge.groupby('loyalty_flag').agg({'prices': ['mean', 'min', 'max']})

prices           
                      mean  min   max
loyalty_flag                         
Loyal customer    7.773559  1.0  25.0
New customer      7.801228  1.0  25.0
Regular customer  7.798274  1.0  25.0

In [32]:
ords_prods_merge['loyalty_flag'].value_counts(dropna = False)

Regular customer    15876776
Loyal customer      10284093
New customer         6243990
Name: loyalty_flag, dtype: int64

#### It now appears that customers are relatively equal in their average spends.

### 6) Create a spending flag for each user based on the average price across all their orders

In [33]:
ords_prods_merge['average_price'] = ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)

In [34]:
ords_prods_merge.head()

,order_id,user_id,order_number,orders_day_of_the_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,_merge,max_order,loyalty_flag,average_price
0,2539329,1,1,2,8,NaN,196,1,0,Soda,77,7,9.0,both,10,New customer,6.367797
1,2398795,1,2,3,7,15.0,196,1,1,Soda,77,7,9.0,both,10,New customer,6.367797
2,473747,1,3,3,12,21.0,196,1,1,Soda,77,7,9.0,both,10,New customer,6.367797
3,2254736,1,4,4,7,29.0,196,1,1,Soda,77,7,9.0,both,10,New customer,6.367797
4,431534,1,5,4,15,28.0,196,1,1,Soda,77,7,9.0,both,10,New customer,6.367797


In [38]:
ords_prods_merge.loc[ords_prods_merge['average_price'] < 10, 'spending_flag'] = 'Low spender'

In [39]:
ords_prods_merge.loc[ords_prods_merge['average_price'] >= 10, 'spending_flag'] = 'High spender'

In [40]:
ords_prods_merge.head()

,order_id,user_id,order_number,orders_day_of_the_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,_merge,max_order,loyalty_flag,average_price,spending_flag
0,2539329,1,1,2,8,NaN,196,1,0,Soda,77,7,9.0,both,10,New customer,6.367797,Low spender
1,2398795,1,2,3,7,15.0,196,1,1,Soda,77,7,9.0,both,10,New customer,6.367797,Low spender
2,473747,1,3,3,12,21.0,196,1,1,Soda,77,7,9.0,both,10,New customer,6.367797,Low spender
3,2254736,1,4,4,7,29.0,196,1,1,Soda,77,7,9.0,both,10,New customer,6.367797,Low spender
4,431534,1,5,4,15,28.0,196,1,1,Soda,77,7,9.0,both,10,New customer,6.367797,Low spender


In [41]:
ords_prods_merge['spending_flag'].value_counts(dropna = False)

Low spender     32285150
High spender      119709
Name: spending_flag, dtype: int64

### 7) Create an order frequency flag that marks the regularity of a user’s ordering behavior according to the median in the “days_since_prior_order” column

In [46]:
ords_prods_merge['median_days'] = ords_prods_merge.groupby(['user_id'])['days_since_prior_order'].transform(np.median)

In [47]:
ords_prods_merge.head()

,order_id,user_id,order_number,orders_day_of_the_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,_merge,max_order,loyalty_flag,average_price,spending_flag,order_frequency,median_days
0,2539329,1,1,2,8,NaN,196,1,0,Soda,77,7,9.0,both,10,New customer,6.367797,Low spender,20.5,20.5
1,2398795,1,2,3,7,15.0,196,1,1,Soda,77,7,9.0,both,10,New customer,6.367797,Low spender,20.5,20.5
2,473747,1,3,3,12,21.0,196,1,1,Soda,77,7,9.0,both,10,New customer,6.367797,Low spender,20.5,20.5
3,2254736,1,4,4,7,29.0,196,1,1,Soda,77,7,9.0,both,10,New customer,6.367797,Low spender,20.5,20.5
4,431534,1,5,4,15,28.0,196,1,1,Soda,77,7,9.0,both,10,New customer,6.367797,Low spender,20.5,20.5


In [49]:
ords_prods_merge.loc[ords_prods_merge['median_days'] > 20, 'order_frequency'] = 'Non-frequent customer'

In [50]:
ords_prods_merge.loc[(ords_prods_merge['median_days'] > 10) & (ords_prods_merge['median_days'] <= 20), 'order_frequency'] = 'Regular customer'

In [51]:
ords_prods_merge.loc[ords_prods_merge['median_days'] <= 10, 'order_frequency'] = 'Frequent customer'

In [52]:
ords_prods_merge['order_frequency'].value_counts(dropna=False)

Frequent customer        21559853
Regular customer          7208564
Non-frequent customer     3636437
NaN                             5
Name: order_frequency, dtype: int64

In [55]:
ords_prods_merge[ords_prods_merge.order_frequency.isnull()]

,order_id,user_id,order_number,orders_day_of_the_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,_merge,max_order,loyalty_flag,average_price,spending_flag,order_frequency,median_days
13645692,895835,159838,1,0,17,NaN,10749,3,0,Organic Red Bell Pepper,83,4,5.7,both,1,New customer,7.42,Low spender,NaN,NaN
17251990,895835,159838,1,0,17,NaN,33401,6,0,Goat Cheese Crumbles,21,16,14.8,both,1,New customer,7.42,Low spender,NaN,NaN
17622767,895835,159838,1,0,17,NaN,23695,2,0,California Veggie Burger,42,1,4.7,both,1,New customer,7.42,Low spender,NaN,NaN
24138593,895835,159838,1,0,17,NaN,21334,5,0,Organic Peeled Garlic,123,4,10.2,both,1,New customer,7.42,Low spender,NaN,NaN
25880002,895835,159838,1,0,17,NaN,22198,1,0,4X Ultra Concentrated Natural Laundry Detergen...,75,17,1.7,both,1,New customer,7.42,Low spender,NaN,NaN


In [56]:
# The NaN values in the order_frequency column are due to new customers who have not yet placed a second order, and therefore their order frequency cannot be calculated

In [58]:
# Replacing the null values in 'order_frequency' to 'First time customer

ords_prods_merge['order_frequency'] = ords_prods_merge['order_frequency'].replace(np.nan, "First time customer")

In [59]:
ords_prods_merge['order_frequency'].value_counts(dropna=False)

Frequent customer        21559853
Regular customer          7208564
Non-frequent customer     3636437
First time customer             5
Name: order_frequency, dtype: int64

In [61]:
# Export data to pkl

ords_prods_merge.to_pickle(os.path.join(path, 'Data','Prepared Data', 'ords_prods_group_agg.pkl'))